In [ ]:
# https://account.sendinblue.com/advanced/api

from __future__ import print_function
import time
import requests
import sib_api_v3_sdk
from sib_api_v3_sdk.rest import ApiException
from pprint import pprint

import csv

# chargement de la clef API via dotenv

from dotenv import load_dotenv
load_dotenv()
SIB_API_KEY = os.environ.get('SIB_API_KEY')
DATA_FOLDER = os.environ.get('DATA_FOLDER')

In [ ]:
# Get all the contacts : https://developers.sendinblue.com/reference/getcontacts-1

def get_all_contacts(apikey, limit = 50, offset = 0, modified_since = '2020-09-20T19:20:30+01:00', sort = 'desc'):
    """Récupère l'intégralité des contacts du compte

    Parameters
    ----------
    apikey : str
        API key
    limit : int64
        Number of documents per page
    offset : int64
        Index of the first document in the page
    modified_since : string (UTC date-time : 'YYYY-MM-DDTHH:mm:ss.SSSZ')
        Filter (urlencoded) the contacts modified after a given UTC date-time (YYYY-MM-DDTHH:mm:ss.SSSZ).
    sort : str ('asc' or 'desc')
        Sort the results in the ascending/descending order of record creation. Default order is descending if sort is not passed

    Returns
    -------
    contacts : Json
        Liste de tous les contacts correspondant à la requête

    """
    configuration = sib_api_v3_sdk.Configuration()
    configuration.api_key['api-key'] = apikey
    api_instance = sib_api_v3_sdk.ContactsApi(sib_api_v3_sdk.ApiClient(configuration))
    try:
        api_response = api_instance.get_contacts(limit=limit, offset=offset, modified_since=modified_since, sort = sort)
        pprint(api_response)
    except ApiException as e:
        print("Exception when calling ContactsApi->get_contacts: %s\n" % e)


get_all_contacts(SIB_API_KEY, modified_since = '2022-01-06T00:00:00+01:00')

In [ ]:
# Get all the lists : https://developers.sendinblue.com/reference/getlists-1

def get_all_lists(apikey, limit = 50, offset = 0):
    """Récupère l'intégralité des listes du compte

    Parameters
    ----------
    apikey : str
        API key
    limit : int64
        Number of documents per page
    offset : int64
        Index of the first document in the page
    sort : str ('asc' or 'desc')
        Sort the results in the ascending/descending order of record creation. Default order is descending if sort is not passed

    Returns
    -------
    count : int64
        Nombre de listes
    lists : Json
        Liste de toutes les listes suivant la requête

    """
    configuration = sib_api_v3_sdk.Configuration()
    configuration.api_key['api-key'] = apikey
    api_instance = sib_api_v3_sdk.ListsApi(sib_api_v3_sdk.ApiClient(configuration))

    try:
        api_response = api_instance.get_lists(limit=limit, offset=offset)
        pprint(api_response)
    except ApiException as e:
        print("Exception when calling ListsApi->get_lists: %s\n" % e)

get_all_lists(SIB_API_KEY)



In [ ]:
# Import contacts : https://developers.sendinblue.com/reference/importcontacts-1

# default listIds = "nouveau_contact"

def import_contacts(apikey, fileBody, listIds):
    """Récupère l'intégralité des listes du compte

    Parameters
    ----------
    apikey : str
        API key
    fileBody : .txt or .csv
        Mandatory if fileUrl is not defined. CSV content to be imported.
        Use semicolon to separate multiple attributes.
        Maximum allowed file body size is 10MB.
        However we recommend a safe limit of around 8 MB to avoid the issues caused due to increase of file body size while parsing.
        Please use fileUrl instead to import bigger files.
    listIds : array of integers
        Mandatory if newList is not defined. Ids of the lists in which the contacts shall be imported.
        For example, [2, 4, 7].

    Returns
    -------

    """

    configuration = sib_api_v3_sdk.Configuration()
    configuration.api_key['api-key'] = apikey

    api_instance = sib_api_v3_sdk.ContactsApi(sib_api_v3_sdk.ApiClient(configuration))
    request_contact_import = sib_api_v3_sdk.RequestContactImport()
    request_contact_import.file_body = fileBody
    request_contact_import.list_ids = listIds
    request_contact_import.email_blacklist = False
    request_contact_import.sms_blacklist = False
    request_contact_import.update_existing_contacts = True
    request_contact_import.empty_contacts_attributes = False

    try:
        api_response = api_instance.import_contacts(request_contact_import)
        pprint(api_response)
    except ApiException as e:
        print("Exception when calling ContactsApi->import_contacts: %s\n" % e)

fileBodyTest = "EMAIL;NOM;PRENOM;SMS\n#john@example.com;John;Doe;31234567923"

import_contacts(SIB_API_KEY, fileBody = fileBodyTest , listIds=[15])


In [75]:
# Save list of all lists to csv

configuration = sib_api_v3_sdk.Configuration()
configuration.api_key['api-key'] = SIB_API_KEY

api_instance = sib_api_v3_sdk.ListsApi(sib_api_v3_sdk.ApiClient(configuration))
limit = 10
offset = 0

try:
    api_response = api_instance.get_lists(limit=limit, offset=offset)
    # pprint(api_response)
except ApiException as e:
    print("Exception when calling ListsApi->get_lists: %s\n" % e)

list_of_lists = api_response.lists

list_file_path = os.path.join('/home/sy/Téléchargements', 'sendinblue_lists' + '.csv')

keys = list_of_lists[0].keys()

with open(list_file_path, 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(list_of_lists)

In [ ]:
# Add a contact to a list : https://developers.sendinblue.com/reference/addcontacttolist-1

configuration = sib_api_v3_sdk.Configuration()
configuration.api_key['api-key'] = SIB_API_KEY

api_instance = sib_api_v3_sdk.ListsApi(sib_api_v3_sdk.ApiClient(configuration))
list_id = 16
contact_emails = sib_api_v3_sdk.AddContactToList()
contact_emails.emails = ['#john@example.com']

try:
    api_response = api_instance.add_contact_to_list(list_id, contact_emails)
    pprint(api_response)
except ApiException as e:
    print("Exception when calling ListsApi->add_contact_to_list: %s\n" % e)

In [ ]:
# Delete a contact from a list : https://developers.sendinblue.com/reference/removecontactfromlist

configuration = sib_api_v3_sdk.Configuration()
configuration.api_key['api-key'] = SIB_API_KEY

api_instance = sib_api_v3_sdk.ListsApi(sib_api_v3_sdk.ApiClient(configuration))
list_id = 16
contact_emails = sib_api_v3_sdk.RemoveContactFromList()
contact_emails.emails = ['#john@example.com']

try:
    api_response = api_instance.remove_contact_from_list(list_id, contact_emails)
    pprint(api_response)
except ApiException as e:
    print("Exception when calling ListsApi->remove_contact_from_list: %s\n" % e)

In [ ]:
# Get all folders : https://developers.sendinblue.com/reference/getfolders-1

configuration = sib_api_v3_sdk.Configuration()
configuration.api_key['api-key'] = SIB_API_KEY

api_instance = sib_api_v3_sdk.ContactsApi(sib_api_v3_sdk.ApiClient(configuration))
limit = 10
offset = 0

try:
    api_response = api_instance.get_folders(limit, offset)
    pprint(api_response)
except ApiException as e:
    print("Exception when calling ContactsApi->get_folders: %s\n" % e)

